## Predictive Insights Project

In [3]:
import pandas as pd

df = pd.read_csv('test_data.csv')

# Display the first few rows of the dataframe
df.head()

,Unnamed: 0,value,test,sub_group
0,1,1.481962,A,1
1,2,1.201471,B,1
2,3,-0.832956,B,1
3,4,0.690690,A,1
4,5,0.266877,A,2
